In [3]:
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.colors as mcolors
from scipy.stats import bootstrap
import random as rd
from file_helper import load_all_files
# sketchy way of figuring out if latex is installed, might fail.
# If it does, comment out
from distutils.spawn import find_executable
from itertools import cycle
import os
import pickle as pk


In [ ]:
path_dir = 'results'
list_files = load_all_files(path_dir)
print(list_files)
for file in list_files:
    file_name = file.split('.')[0]
    if 'binning' in file_name:
        print(file_name)
        with open(os.path.join(path_dir, file), 'rb') as f:  # load the data
                data = pk.load(f)

In [5]:
baseline_name = list(data['data'].keys())
baseline_name

['argmaxAR', 'FCDM', 'transformer', 'ground truth']

In [14]:
data['data'][baseline_name[1]][1]

[{'p': [0.74716765874526, 0.0028323412698408935, 0.24960413912009188, 0],
  'q': [0, 0.6867999999999409, 0, 0.025500000000074463]},
 {'p': [0.7471978340098131, 0.002802166005290644, 0.24959587192432392, 0],
  'q': [0, 0.6790999999999416, 0, 0.027500000000074354]}]